In [1]:
from fastsam import FastSAM, FastSAMPrompt
import torch

In [3]:
model = FastSAM(r'/mnt/c/Users/gtraw/Projekt_badawczy/checkpoints/FASTSAM/FastSAM-s.pt')
IMAGE_PATH = r'/mnt/c/Users/gtraw/Projekt_badawczy/images/weti1_2024_11_21_14_55.jpg'
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
everything_results = model(IMAGE_PATH, device=DEVICE, retina_masks=True, imgsz=1024, conf=0.5, iou=0.9,)
prompt_process = FastSAMPrompt(IMAGE_PATH, everything_results, device=DEVICE)

# everything prompt
ann = prompt_process.everything_prompt()

prompt_process.plot(annotations=ann, output_path=r'/mnt/c/Users/gtraw/Projekt_badawczy/results/fastsams_zima.jpg',)

/mnt/c/Users/gtraw/Projekt_badawczy/FastSAM/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # l

In [10]:
ann.cpu().numpy()

array([[[          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0],
        ...,
        [          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0]],

       [[          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,           0, ...,           0,           0,           0],
        ...,
        [          0,           0,           0, ...,           0,           0,           0],
        [          0,           0,        

### Segmentacja na całym zbiorze

In [6]:
from fastsam import FastSAM, FastSAMPrompt
import torch

import cv2
import sys
import os
import time
import json

import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

model_info = {
    'fastsam_x': "/mnt/c/Users/gtraw/Projekt_badawczy/checkpoints/FASTSAM/FastSAM-x.pt",
    'fastsam_s': "/mnt/c/Users/gtraw/Projekt_badawczy/checkpoints/FASTSAM/FastSAM-s.pt"
}

# Images/results paths
input_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_12'
output_dir = '/mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_12/dataset_results'

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
def transform_to_polygons(data, mode="points"):
    polygons = []
    if mode == "points":
        polygons = [points2polygon(d) for d in data]
    elif mode == "corners":
        polygons = [corners2polygon(d) for d in data]
    elif mode == "mask":
        polygons = [mask2polygon(d) for d in data]
    return [p for p in polygons if p is not None]
    
def corners2polygon(corners):
    x1, y1, x2, y2 = corners
    points = [(x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)]
    
    return Polygon(points)
        
def points2polygon(points):
    return Polygon(points)

def mask2polygon(mask):
    mask = np.array(mask)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    
    # if len(largest_contour) < 4:
    #     print(f"< 4 points: {largest_contour=}")
    #     return None  # Not enough points for a valid polygon
    
    try:
        largest_contour = max(contours, key=cv2.contourArea)
    except Exception as e:
        print(f"contour exception: {e}")
        plt.figure(figsize=(8, 8))
        plt.imshow(mask, cmap='gray')
        if contour is not None:
            # Plot contour points
            contour = contour.squeeze()
            plt.plot(contour[:, 0], contour[:, 1], 'r-', linewidth=2)
        plt.title("Invalid Polygon Mask")
        plt.axis("off")
        plt.show()

    polygon = Polygon(shell=largest_contour.squeeze())
    
    if not polygon.is_valid:
        polygon = polygon.buffer(0)
    
    return polygon

def iou(polygon1, polygon2):
    if not polygon1.is_valid:
        polygon1 = polygon1.buffer(0)
    if not polygon2.is_valid:
        polygon2 = polygon2.buffer(0)
    union = polygon1.union(polygon2).area
    if union <= 0:
        return 0
    intersection = polygon1.intersection(polygon2).area
    return intersection / union

def build_similarity_matrix(objects_db, objects_pred):
    len_db, len_pred = len(objects_db), len(objects_pred)
    print(f"Matrix size: [{len_db}, {len_pred}]")
    similarity_matrix = np.zeros([len_db, len_pred])
    
    time_start = time.time()
    for x, obj_db in enumerate(objects_db):
        for y, obj_pred in enumerate(objects_pred):
            similarity_matrix[x, y] = iou(obj_db, obj_pred)
            print(f"Building similarity matrix: {x * len_pred + y + 1:>10}/{len_db*len_pred}", end='\r')
    time_end = time.time() - time_start
    print(f"Similarity matrix built in {time_end:.2f}s" + ' ' * 100)
    return similarity_matrix

def match_polygons(similarity_matrix):
    xs, ys = linear_sum_assignment(similarity_matrix, maximize=True)
    matching = [[x, y, similarity_matrix[x, y]] for x, y in zip(xs, ys) if similarity_matrix[x][y] > 0]
    return matching

def evaluate_segmentation(polygons_db, polygons_pred, matching, time, weather_info = [], model_info = []):
    results = {
        "weather": weather_info,
        "model": model_info,
        "time": time
    }
    num_db = len(polygons_db)
    num_pred = len(polygons_pred)
    num_matching = len(matching)

    results['ground_truth_objects'] = num_db
    results['segmented_objects'] = num_pred
    results['matched_objects'] = num_matching

    if num_pred != 0:
        results['precision'] = num_matching / num_pred
    else:
        results['precision'] = 0

    if num_db != 0:
        results['recall'] = num_matching / num_db
    else:
        results['recall'] = 0

    if results['precision'] + results['recall'] != 0:
        results['f1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall'])
    else:
        results['precision'] = 0

    ious = []
    for x, y, _ in matching:
        ious.append(iou(polygons_db[x], polygons_pred[y]))

    if num_matching != 0:
        results['mean_iou'] = sum(ious) / num_matching
    else:
        results['mean_iou'] = 0

    return results

In [7]:
for model_name in model_info.keys():
    model = FastSAM(model_info[model_name])
    output_dir_model = f"{output_dir}_{model_name}"
    print(f"output_dir: {output_dir_model}")

    # fix directories
    if not os.path.exists(output_dir_model):
        os.makedirs(output_dir_model)

    # for image in dir
    for image_file in os.listdir(input_dir):
        data_pred = {
            'model': model_name
        }

        # skip non-images
        if not image_file.endswith(('jpg', 'png')):
            continue

        print(f"\nImage: {image_file}")
        # image's path
        image_path = os.path.join(input_dir, image_file)
        # measure time
        time_start = time.time()
        # generate masks
        everything_results = model(image_path, device=DEVICE, imgsz=1920)
        prompt_process = FastSAMPrompt(image_path, everything_results, device=DEVICE)

        # everything prompt
        ann = prompt_process.everything_prompt()
        time_end = time.time() - time_start

        # save results
        data_pred['time'] = time_end
        data_pred['objects'] = []

        ann = ann.cpu()
        for seg in ann:
            data_pred['objects'].append(seg)

        # load labeled data
        data_filename = '.'.join(image_file.replace('_m', '').split('.')[:-1]) + '.json'
        with open(os.path.join(input_dir, data_filename), 'r') as f:
            data_db = json.load(f)

        data_db['polygons'] = transform_to_polygons([obj['points'] for obj in data_db['objects']])
        # print(data_pred['objects'][-1])
        # print(data_pred['objects'][-1].shape)
        data_pred['polygons'] = transform_to_polygons([obj for obj in data_pred['objects']], "mask")

        sm = build_similarity_matrix(data_db['polygons'], data_pred['polygons'])
        matching = match_polygons(sm)

        result_dict = evaluate_segmentation(data_db['polygons'], data_pred['polygons'], matching, data_pred['time'], data_db['weather'], ['sam2.1', model_name])
        result_dict['filename'] = '.'.join(image_file.split('.')[:-1])

        json_path = os.path.join(output_dir_model, f"{'.'.join(image_file.split('.')[:-1])}.json")
        with open(json_path, 'w') as json_file:
            json.dump(result_dict, json_file, indent=4)

/mnt/c/Users/gtraw/Projekt_badawczy/FastSAM/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # l

output_dir: /mnt/c/Users/gtraw/Projekt_badawczy/dataset_results_11_12/dataset_results_fastsam_x

Image: weti1_2024_11_12_09_28.jpg



image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_12/weti1_2024_11_12_09_28.jpg: 1088x1920 236 objects, 542.5ms
Speed: 16.9ms preprocess, 542.5ms inference, 857.7ms postprocess per image at shape (1, 3, 1920, 1920)


Matrix size: [9, 236]


Similarity matrix built in 0.25s                                                                                                    

Image: weti1_2024_11_12_09_28_m.jpg


...

Matrix size: [0, 4]
Similarity matrix built in 0.00s                                                                                                    

Image: weti2_2024_11_12_18_34.jpg


image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_12/weti2_2024_11_12_18_34.jpg: 1088x1920 11 objects, 44.7ms
Speed: 5.3ms preprocess, 44.7ms inference, 4.5ms postprocess per image at shape (1, 3, 1920, 1920)



Matrix size: [0, 11]
Similarity matrix built in 0.00s                                                                                                    

Image: weti2_2024_11_12_18_34_m.jpg


image 1/1 /mnt/c/Users/gtraw/Projekt_badawczy/dataset_11_12/weti2_2024_11_12_18_34_m.jpg: 1088x1920 4 objects, 42.8ms
Speed: 6.4ms preprocess, 42.8ms inference, 4.1ms postprocess per image at shape (1, 3, 1920, 1920)


Matrix size: [0, 4]
Similarity matrix built in 0.00s                                                                                                    
